In [1]:
import talib
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [12]:
class BackTest:
    
    def __init__(self, instrument):
        
#         self.df = pd.DataFrame(yf.download(tickers=instrument, start = '2022-10-01', end = '2023-06-01', interval='1h'))
        if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
        login = 114781990
        password = 'Mypassword$1234'
        server = 'Exness-MT5Trial6'

        mt5.login(login, password, server)
        print('Working on: ', instrument[0])
        rate = mt5.copy_rates_from(instrument[0], mt5.TIMEFRAME_M15, datetime.datetime.now(), instrument[2])
        self.df = pd.DataFrame(rate)
        
         
        self.df['date']=pd.to_datetime(self.df['time'], unit='s')

        
        self.analyse = pd.DataFrame(columns=['TOTAL TRADES', 'WINS', 'LOSSES', 'PERFORMANCE', 'WIN RATE', 
                                'WINNING STREAK', 'LOSING STREAK', 'MAX DRAWDOWN', 'AVG GAIN', 'AVG LOSS'])
        
        '''Renaming columns and deleting unnecessary columns'''

        self.df.set_index(np.arange(len(self.df)), inplace = True)
        self.df = self.df.drop(['spread', 'real_volume', 'tick_volume'], axis='columns')
        
        
        self.one_pip_value = instrument[1]
        self.account = 10000
#         print(self.df.head())
        
     # KELTNER CHANNEL CALCULATION

    def get_kc(self, high, low, close, kc_lookback, multiplier, atr_lookback):
        tr1 = pd.DataFrame(high - low)
        tr2 = pd.DataFrame(abs(high - close.shift()))
        tr3 = pd.DataFrame(abs(low - close.shift()))
        
        # Calculate True Range as the element-wise maximum of the three components
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.ewm(alpha=1/atr_lookback).mean()

        kc_middle = close.ewm(span=kc_lookback).mean()
        kc_upper = kc_middle + multiplier * atr
        kc_lower = kc_middle - multiplier * atr

        return kc_middle, kc_upper, kc_lower
    
    
    def indicators(self):
        '''Adding EMAs, BB, Keltner Channel and RSI'''
        
        self.df['EMA_8'] = talib.EMA(self.df['close'], timeperiod=8)

        # EMA 20 group
        self.df['EMA_15'] = talib.EMA(self.df['close'], timeperiod=15)
        self.df['EMA_200'] = talib.EMA(self.df['close'], timeperiod=200)


        self.df['RSI'] = talib.RSI(self.df['close'], timeperiod=21)
        self.df = self.df.dropna()

        self.df.set_index(np.arange(len(self.df)), inplace = True)
        # Identify trend on 21 days highs & lows

        '''Get Keltner for trading'''
        
        self.df['EMA_20'],self.df['upper_keltner'],self.df['lower_keltner'] = self.get_kc(self.df['high'], self.df['low'], self.df['close'], kc_lookback = 20, multiplier = 2.25, atr_lookback = 10)
        
        self.df = self.df.dropna()

        self.df.set_index(np.arange(len(self.df)), inplace = True)

        self.df.index.name='sn'
        # print('Updated DF: ',self.df.head())
        
        
    def candle(self, df):
        '''Return candle colour'''

        if df['close'] - df['open'] > 0:
            return 'G'

        else:
            return 'R'
        
        
    def mark_emas(self):
        '''Marking candles with colour'''
        
        self.df['candle'] = self.df.apply(self.candle, axis=1)
        
        '''Marking candles on emas'''
        
        self.df['pos_trade'] = self.df.apply(lambda x: '1' if  (
                               
                                ( x['high'] >= x['EMA_15'] and x['EMA_15']> x['close']) or
            
                                (x['low'] <= x['EMA_15'] and x['EMA_15'] < x['close'])
                                )
                                else '0', axis=1)
        

#     calculate climax with
#     RSI OB/OS and candle body outside keltner's channel extreme
        
        count_candles = 0
        
#       
        self.df['tradable'] = True
    
#     For counting 20 candles after climax

        buy_flag = True
        sell_flag = True
        green_count = 0
        red_count = 0
        
        self.df['tradable'] = 0
        
#         To block next 20 candles starting candle must
#         be atleast 0.07% away from channel extreme
#         calculate diff(close[outside candle], channel_extreme) >= 0.07
        
        for index, row in self.df.iterrows():
            
            one_percent = row['open']/100 
            
#             When green candle goes outside channel
            if ( ((row['candle'] == 'G' and row['open'] > row['upper_keltner']) or
                
                (row['candle'] == 'R' and row['close'] > row['upper_keltner'])) and
                
                sell_flag == True and 
                
                ( abs(row['close'] - row['upper_keltner'])/one_percent ) >= 0.07 ):
        
                
                buy_flag = False
                green_count = 0
                self.df.at[index,'tradable'] = 'First G Break'
                
#             When price remain outside after green breakout
            elif( row['close'] > row['upper_keltner'] and sell_flag == True and 
                 buy_flag == False and green_count == 0):
                
                self.df.at[index,'tradable'] = 'Outside after G Break'
                

#            Any candle in 20 count closes outside channel, restart the count

            elif row['close'] > row['upper_keltner'] and buy_flag == True and green_count > 0:
                
                green_count = 0
                buy_flag = False
                self.df.at[index,'tradable'] = 'Outside after G Break'
            
#             When price returns inside channel after buy breakout
            elif row['close'] < row['upper_keltner'] and buy_flag == False:
                
                buy_flag = True
                green_count = 1
                self.df.at[index,'tradable'] = 'G count 1'
                
#                 Counting 20 candles when green breakout moves inside channel
            elif green_count > 0 and green_count < 19 and row['close'] < row['upper_keltner']:
                
                green_count += 1
                self.df.at[index,'tradable'] = 'Counting after G Break'
               
#             When 20 candles after climax are over
            elif green_count == 19 and row['close'] < row['upper_keltner'] and red_count == 0:
                
                green_count = 0
                self.df.at[index,'tradable'] = 0
                
                
#                 =============================================
                
#             When red candle goes outside channel
            if ( ((row['candle'] == 'R' and row['open'] < row['lower_keltner'] ) or
                 
                (row['candle'] == 'G' and row['close'] < row['lower_keltner']) )and 
                
                buy_flag == True  and
                
                ( abs(row['lower_keltner'] - row['close'])/one_percent ) >= 0.07 ):
                
                sell_flag = False
                red_count = 0
                self.df.at[index,'tradable'] = 'Red Breakout'
                
#             When price remain outside channel after red breakout

            elif row['close'] < row['lower_keltner'] and buy_flag == True and sell_flag == False:
                
                self.df.at[index,'tradable'] = 'Outside after Red Breakout'
                
                
#            Any candle in 20 count closes outside channel, restart the count

            elif row['close'] < row['lower_keltner'] and sell_flag == True and red_count > 0:
                
                red_count = 0
                sell_flag = False
                self.df.at[index,'tradable'] = 'Outside after R Break'
                

#             When price returns inside channel after sell breakout
            elif row['close'] > row['lower_keltner'] and sell_flag == False:
                
                sell_flag = True
                red_count = 1
                self.df.at[index,'tradable'] = 'Sell count 1'
                
#                 Counting 20 candles when red breakout moves inside channel
            elif red_count > 0 and red_count < 19 and row['close'] > row['lower_keltner']:
                
                red_count += 1
                self.df.at[index,'tradable'] = 'Counting after sell breakout'
               
#             When 20 candles after climax are over
            elif red_count == 19 and row['close'] > row['lower_keltner'] and green_count == 0:
                
                red_count = 0
                self.df.at[index,'tradable'] = 0

        self.df.to_excel('Check_climax.xlsx')
        

In [13]:
currency = BackTest(['USDJPYm', 6.82682, 24000])
currency.indicators()
currency.mark_emas()

Working on:  USDJPYm
